In [1]:
!pip3 install findspark

In [2]:
import findspark
findspark.init()

In [3]:
import os
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [4]:
#executing the shell.py inside the 'SPARK_HOME' environment

exec(open(os.path.join(os.environ["SPARK_HOME"],'python/pyspark/shell.py')).read())

:: loading settings :: url = jar:file:/Users/tania/Downloads/spark/spark-3.3.2-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/tania/.ivy2/cache
The jars for the packages stored in: /Users/tania/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-49f8046d-3e2d-4256-b247-974f3209c50d;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.3.2 in central
	found org.tukaani#xz;1.9 in central
	found org.spark-project.spark#unused;1.0.0 in central
:: resolution report :: resolve 412ms :: artifacts dl 12ms
	:: modules in use:
	org.apache.spark#spark-avro_2.12;3.3.2 from central in [default]
	org.spark-project.spark#unused;1.0.0 from central in [default]
	org.tukaani#xz;1.9 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     

23/08/08 19:47:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.3.2
      /_/

Using Python version 3.9.13 (main, Aug 25 2022 18:29:29)
Spark context Web UI available at http://192.168.0.25:4040
Spark context available as 'sc' (master = local[*], app id = local-1691534859990).
SparkSession available as 'spark'.


# Creating Dataframe

## Example 1

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
spark.read.json('/Users/tania/Downloads/spark/Spark-Programming-In-Python-master/notebook/data/people.json').show()

+----+--------+
| age|    name|
+----+--------+
|null|Prashant|
|  30|   Abdul|
|  19|  Justin|
|  43|    Andy|
+----+--------+



In [6]:
def to_date_df(df, fmt, field):
    return df.withColumn(field, to_date(col(field), fmt))

In [7]:
my_schema = StructType([
    StructField('ID', StringType()),
    StructField('EventDate', StringType())
])

my_rows = [
    Row('1', '4/5/2020'),
    Row('2', '08/15/2022'),
    Row('3', '01/01/2023'),
    Row('4', '12/31/2022')
]

my_rdd = spark.sparkContext.parallelize(my_rows)
my_df = spark.createDataFrame(my_rdd, my_schema)
my_df.show()

+---+----------+
| ID| EventDate|
+---+----------+
|  1|  4/5/2020|
|  2|08/15/2022|
|  3|01/01/2023|
|  4|12/31/2022|
+---+----------+



In [64]:
new_df = to_date_df(my_df, 'M/d/y', 'EventDate')
new_df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- EventDate: date (nullable = true)



In [65]:
new_df.show()

+---+----------+
| ID| EventDate|
+---+----------+
|  1|2020-04-05|
|  2|2022-08-15|
|  3|2023-01-01|
|  4|2022-12-31|
+---+----------+



## Example 2

In [48]:
data_list = [
    ('Tania', 3,11,1987),
    ('Daniel', 12,12,1980),
    ('Juliana', 31,1,1983),
    ('Pedro', 31,1,84),
    ('Pedro', 31,1,84),
    ('Maria', 15,5,1) # duplicate
]

raw_df = spark.createDataFrame(data_list).toDF('Name', 'Day', 'Month', 'Year')
raw_df.show()

+-------+---+-----+----+
|   Name|Day|Month|Year|
+-------+---+-----+----+
|  Tania|  3|   11|1987|
| Daniel| 12|   12|1980|
|Juliana| 31|    1|1983|
|  Pedro| 31|    1|  84|
|  Pedro| 31|    1|  84|
|  Maria| 15|    5|   1|
+-------+---+-----+----+



In [50]:
# cleaning data
raw_df_cleaned = raw_df.withColumn('Year', 
expr(
'''case 
     when Year < 23 then cast(Year as int) + 2000
     when Year <100 then cast(Year as int) + 1900
     else Year 
end'''))
raw_df_cleaned.show()

+-------+---+-----+----+
|   Name|Day|Month|Year|
+-------+---+-----+----+
|  Tania|  3|   11|1987|
| Daniel| 12|   12|1980|
|Juliana| 31|    1|1983|
|  Pedro| 31|    1|1984|
|  Pedro| 31|    1|1984|
|  Maria| 15|    5|2001|
+-------+---+-----+----+



In [32]:
raw_df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Day: long (nullable = true)
 |-- Month: long (nullable = true)
 |-- Year: long (nullable = true)



# Creating new columns 

## with expressions

In [70]:
from pyspark.sql.functions import year, month, dayofmonth

new_df2 = new_df.select('EventDate', \
                       year('EventDate').alias('Year'), \
                       month('EventDate').alias('Month'), \
                       dayofmonth("EventDate").alias('day') )
new_df2.show()

+----------+----+-----+---+
| EventDate|Year|Month|day|
+----------+----+-----+---+
|2020-04-05|2020|    4|  5|
|2022-08-15|2022|    8| 15|
|2023-01-01|2023|    1|  1|
|2022-12-31|2022|   12| 31|
+----------+----+-----+---+



In [74]:
new_df2.withColumn('Birthday', expr('''to_date(concat(Year, '-', Month, '-', day))''')).show()

+----------+----+-----+---+----------+
| EventDate|Year|Month|day|  Birthday|
+----------+----+-----+---+----------+
|2020-04-05|2020|    4|  5|2020-04-05|
|2022-08-15|2022|    8| 15|2022-08-15|
|2023-01-01|2023|    1|  1|2023-01-01|
|2022-12-31|2022|   12| 31|2022-12-31|
+----------+----+-----+---+----------+



In [79]:
new_df2.withColumn('Birthday', to_date(expr('''concat(Year, '-', Month, '-', day)'''))).show()

+----------+----+-----+---+----------+
| EventDate|Year|Month|day|  Birthday|
+----------+----+-----+---+----------+
|2020-04-05|2020|    4|  5|2020-04-05|
|2022-08-15|2022|    8| 15|2022-08-15|
|2023-01-01|2023|    1|  1|2023-01-01|
|2022-12-31|2022|   12| 31|2022-12-31|
+----------+----+-----+---+----------+



## drop column

In [81]:
new_df2.drop('day', 'Month').show()

+----------+----+
| EventDate|Year|
+----------+----+
|2020-04-05|2020|
|2022-08-15|2022|
|2023-01-01|2023|
|2022-12-31|2022|
+----------+----+



## drop duplicates

In [84]:
new_df2.drop_duplicates(['Year']).show()

+----------+----+-----+---+
| EventDate|Year|Month|day|
+----------+----+-----+---+
|2020-04-05|2020|    4|  5|
|2022-08-15|2022|    8| 15|
|2023-01-01|2023|    1|  1|
+----------+----+-----+---+



## create a unique id column

In [30]:
from pyspark.sql.functions import monotonically_increasing_id

raw_df_partitioned = raw_df.repartition(2)
raw_df_partitioned_with_id = raw_df_partitioned.withColumn('id', monotonically_increasing_id())
raw_df_partitioned_with_id.show()

+-------+---+-----+----+----------+
|   Name|Day|Month|Year|        id|
+-------+---+-----+----+----------+
|Juliana| 31|    1|1983|         0|
|  Pedro| 31|    1|  84|         1|
|  Tania|  3|   11|1987|8589934592|
| Daniel| 12|   12|1980|8589934593|
+-------+---+-----+----+----------+



## sort column

In [110]:
new_df2.sort(expr('Month')).show()

+----------+----+-----+---+
| EventDate|Year|Month|day|
+----------+----+-----+---+
|2023-01-01|2023|    1|  1|
|2020-04-05|2020|    4|  5|
|2022-08-15|2022|    8| 15|
|2022-12-31|2022|   12| 31|
+----------+----+-----+---+



In [107]:
new_df2.orderBy('Year').show()

+----------+----+-----+---+
| EventDate|Year|Month|day|
+----------+----+-----+---+
|2020-04-05|2020|    4|  5|
|2022-08-15|2022|    8| 15|
|2022-12-31|2022|   12| 31|
|2023-01-01|2023|    1|  1|
+----------+----+-----+---+



## Substring Column

In [85]:
df = spark.createDataFrame([('a.b.c.d',)], ['s'])
df.select(substring_index(df.s, '.', 1).alias('s')).collect()
df.select(substring_index(df.s, '.', -3).alias('s')).collect()

[Row(s='b.c.d')]

## Inline Column Type Cast

In [51]:
# with cast() as expression
raw_df.withColumn('Year', 
expr(
'''case 
     when Year < 23 then cast(Year as int) + 2000
     when Year <100 then cast(Year as int) + 1900
     else Year 
end''')).show()

+-------+---+-----+----+
|   Name|Day|Month|Year|
+-------+---+-----+----+
|  Tania|  3|   11|1987|
| Daniel| 12|   12|1980|
|Juliana| 31|    1|1983|
|  Pedro| 31|    1|1984|
|  Pedro| 31|    1|1984|
|  Maria| 15|    5|2001|
+-------+---+-----+----+



In [56]:
raw_df.withColumn('Year', \
                  when(col('Year') < 23, col('Year') + 2000) \
                  .when(col('Year') < 100, col('Year') + 1900)\
                  .otherwise(col('Year'))
                 ).show()

+-------+---+-----+----+
|   Name|Day|Month|Year|
+-------+---+-----+----+
|  Tania|  3|   11|1987|
| Daniel| 12|   12|1980|
|Juliana| 31|    1|1983|
|  Pedro| 31|    1|1984|
|  Pedro| 31|    1|1984|
|  Maria| 15|    5|2001|
+-------+---+-----+----+



In [53]:
# with cast() function
raw_df_cleaned = raw_df.withColumn('Year', 
expr(
'''case 
     when Year < 23 then Year + 2000
     when Year <100 then Year + 1900
     else Year 
end''').cast(IntegerType()))
raw_df_cleaned.show()

+-------+---+-----+----+
|   Name|Day|Month|Year|
+-------+---+-----+----+
|  Tania|  3|   11|1987|
| Daniel| 12|   12|1980|
|Juliana| 31|    1|1983|
|  Pedro| 31|    1|1984|
|  Pedro| 31|    1|1984|
|  Maria| 15|    5|2001|
+-------+---+-----+----+

